In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from obspy.core import read, Stream

In [2]:
sw4_path = '/Users/sydneydybing/SW4/strain/'
real_path = '/Users/sydneydybing/StrainProject/2024/'

In [3]:
# Stations

sta_file = 'sw4_strain_stations_metadata.csv'
stas = pd.read_csv(sw4_path + sta_file)
stas

,BNUM,NAME,LAT,LONG,ELEV(m),INSTALL_DATE,CH0(EofN),BSM_Depth(m),SEISMOMETER_Depth(m),PORE_DEPTH(m),...,DATA_END,GAP(m),L_DATE,L0(cnts),L1(cnts),L2(cnts),L3(cnts),REGION,hypo_dist_km,p_arrival
0,B072,goldhl072bcn2007,35.83100,-120.34500,397.7,8/27/07,219.7,159.7,154.5,NaN,...,Present,0.0001,2007:253,47592395,52739856,51898099,51846814,Parkfield_California,248.267863,36.599810
1,B079,jackcn079bcs2006,35.71570,-120.20570,436.6,10/13/06,353.7,181.5,175.1,45.00,...,Present,0.0001,2006:318,48042444,48981765,47787315,49995315,Parkfield_California,235.829519,35.074175
2,B082,pathfi082bcs2006,33.59818,-116.59600,1374.8,6/10/06,324.8,242.6,220.7,22.60,...,Present,0.0002,2006:335,49597819,53081549,47839224,49741765,Anza_California,257.821320,37.771597
3,B087,fordra087bcs2006,33.49550,-116.60267,1139.0,6/17/06,340.7,160.9,147.5,24.38,...,Present,0.0002,2006:335,49940592,51287823,50044783,49977525,Anza_California,268.302260,39.057145
4,B916,marips916bcs2008,36.19250,-117.66850,1859.9,3/13/08,303.6,176.5,166.9,NaN,...,Present,0.0001,2008:090,55948375,53191200,55522127,49391430,Mojave_California,47.348199,8.224188
5,B917,tonyso917bcs2008,35.38920,-117.25870,1125.9,7/14/08,306.1,162.5,Unknown,NaN,...,Present,0.0001,2008:219,51200216,53318503,50164819,49964400,Mojave_California,52.284362,9.024224
6,B918,mtsprn918bcs2008,35.93570,-117.60170,1042.6,6/21/08,288.9,196.7,189.9,NaN,...,Present,0.0001,2008:245,56324863,55262353,53608304,55388642,Mojave_California,18.442171,3.584549
7,B921,randsb921bcs2008,35.58650,-117.46220,694.5,6/3/08,340.5,147.4,139.9,NaN,...,Present,0.0001,2008:178,53324139,53302937,55141972,53475662,Mojave_California,23.798922,4.428801


In [8]:
# Subplot version

count = 0
fig = plt.figure(figsize = (8,8), dpi = 400, facecolor = 'white')

for idx in range(len(stas)):
    
    sta = stas.BNUM.values[idx]
    theo_p_arr = stas.p_arrival.values[idx] # Seconds after origin time of earthquake
    hypo_dist = stas.hypo_dist_km.values[idx]
    
    # Noisy filtered SW4 data
    sw4_peak_strain = np.load(sw4_path + 'final_RC_model_outputs_processed/strain/noise_added/filtered/peak/' + str(sta) + '.npy')
    
    try:
        real_peak_strain = read(real_path + 'strain_data/3comp_peak_post_p/2019-07-06_M7.1/' + sta + '.mseed')
    except:
        print('No real data for station ' + sta)
        continue
    
    count += 1
    
    sw4_peak_strain_times = sw4_peak_strain[:,0]
    sw4_peak_strain_data = sw4_peak_strain[:,1]
    
    sw4_max_time = max(sw4_peak_strain_times)
    sw4_npts = len(sw4_peak_strain_times)
    sw4_samp_rate = sw4_npts / sw4_max_time
    
    xlow = 5
    xhigh = 40
    
    real_peak_strain_times = real_peak_strain[0].times() # This has been shifted so that the P-wave arrival is at 10 seconds
    real_peak_strain_data = real_peak_strain[0].data
    
    # Shift the SW4 peak strain so that it lines up with the real peak strain, which has the P-wave arrival at 10 seconds
    
    if theo_p_arr >= 10: # Shift the peak strain backward
        shift_amount = theo_p_arr - 10
        # sw4_rms_strain_times = sw4_rms_strain_times - shift_amount
        sw4_peak_strain_times = sw4_peak_strain_times - shift_amount
    else: # Shift the peak strain forward
        shift_amount = 10 - theo_p_arr
        # sw4_rms_strain_times = sw4_rms_strain_times + shift_amount
        sw4_peak_strain_times = sw4_peak_strain_times + shift_amount

    ax = fig.add_subplot(3,2,count)
    plt.grid(alpha = 0.5, zorder = 0)
    
    ax.plot(real_peak_strain_times, real_peak_strain_data, color = 'red', label = 'Real peak strain')
    ax.plot(sw4_peak_strain_times, sw4_peak_strain_data, color = 'blue', label = 'SW4 peak strain')
    ax.semilogy()
    
    ax.axvline(10, color = 'gray', linestyle = '--', label = 'P-wave arrival')
    
    ax.set_xlim(xlow, xhigh)
    if sta == 'B072' or sta == 'B079':
        ax.set_ylim(min(sw4_peak_strain_data[int(xlow*sw4_samp_rate):int(xhigh*sw4_samp_rate)])*0.8, 10**-6)
        
    ax.text(x = 0.65, y = 0.1, s = sta, fontsize = 20, transform = ax.transAxes) 
    ax.set_ylabel('Peak strain', fontsize = 14)
    
    if count == 1:
        ax.text(x = -0.37, y = 0.9, s = '(a)', fontsize = 20, transform = ax.transAxes)
    elif count == 2:
        ax.text(x = -0.37, y = 0.9, s = '(b)', fontsize = 20, transform = ax.transAxes)
    elif count == 3:
        ax.text(x = -0.37, y = 0.9, s = '(c)', fontsize = 20, transform = ax.transAxes)
        ax.legend(bbox_to_anchor = [2.2, -0.3])
    elif count == 4:
        ax.text(x = -0.37, y = 0.9, s = '(d)', fontsize = 20, transform = ax.transAxes)
    elif count == 5:
        ax.text(x = -0.37, y = 0.9, s = '(e)', fontsize = 20, transform = ax.transAxes)
    if count == 1 or count == 2 or count == 3:
        ax.tick_params(axis = 'both', bottom = False, labelbottom = False)
    elif count == 4 or count == 5:
        ax.set_xlabel('Time (s)', fontsize = 14)
    
plt.subplots_adjust(hspace = 0, wspace = 0.42)

# plt.show()

# plt.savefig('/Users/sydneydybing/StrainProject/2024/Manuscript/Figures/renamed_figs/Figure_8.jpg', format = 'JPG')
plt.savefig('/Users/sydneydybing/dissertation/defense_figs/strain/Figure_8.png', format = 'PNG')
plt.close();

No real data for station B082
No real data for station B087
No real data for station B918
